# ETL Pipeline - Cittamobi ML Project

Este notebook contém o processo completo de ETL (Extract, Transform, Load) para preparação dos dados do projeto de Machine Learning da Cittamobi.

## Etapas do Pipeline:
1. **Importação de Bibliotecas**
2. **Carregamento de Dados** (dataset_completo.csv + GTFS)
3. **Limpeza e Tratamento de Dados**
4. **Extração de Coordenadas**
5. **Engenharia de Features**
   - Features Temporais
   - Features Geoespaciais
   - Features de Serviço (GTFS)
   - Features de Agregação
   - Features Cíclicas e de Interação
6. **Criação da Variável Alvo** (lotacao_proxy_binaria)
7. **Exportação da Tabela Final**

## 1. Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import holidays
from geopy.distance import geodesic
from scipy.spatial import cKDTree
from sklearn.preprocessing import LabelEncoder
import warnings
import re

# Ignorar avisos para manter a saída limpa
warnings.filterwarnings('ignore')

# Configurar pandas para mostrar mais colunas
pd.set_option('display.max_columns', None)

print("✅ Bibliotecas importadas com sucesso.")

## 2. Configurações e Funções Auxiliares

In [ ]:
# Fuso horário de São Paulo
TZ = 'America/Sao_Paulo'

# Lista de feriados no Brasil (SP)
br_holidays = holidays.Brazil(state='SP')

def extract_coords(point_str):
    """Extrai latitude e longitude de uma string POINT(-lon -lat)"""
    if pd.isna(point_str):
        return None, None
    
    # Regex para encontrar os dois números (longitude e latitude)
    match = re.search(r'POINT\s*\(\s*(-?\d+\.?\d*)\s+(-?\d+\.?\d*)\s*\)', str(point_str))
    if match:
        # A ordem no POINT é (longitude, latitude)
        lon = float(match.group(1))
        lat = float(match.group(2))
        return lat, lon
    return None, None

print("✅ Constantes e funções auxiliares definidas.")

## 3. Carregamento de Dados

### 3.1 Dataset Principal

In [ ]:
# Carregar o dataset principal
try:
    df_events = pd.read_csv('dataset_completo.csv')
    
    df_events['event_timestamp'] = pd.to_datetime(df_events['event_timestamp'], format='mixed').dt.tz_convert(TZ)
    
    # Filtrar apenas eventos relevantes
    df_events = df_events[df_events['event_name'] == 'bstop_open'].copy()
    
    # Remover colunas que não parecem úteis para o modelo
    cols_to_drop = ['event_date', 'ga_session_id', 'event_name', 'platform', 
                    'operating_system_version', 'mobile_brand_name', 'mobile_model_name', 
                    'mobile_marketing_name', 'traffic_source_source', 'ctm_country', 
                    'ctm_state', 'ctm_city', 'ctm_district', 'stop_address',
                    'ctm_stop_fav', 'ctm_service_fav', 'headsign']
    
    # Filtrar colunas que realmente existem no DataFrame antes de dropar
    existing_cols_to_drop = [col for col in cols_to_drop if col in df_events.columns]
    df_events.drop(columns=existing_cols_to_drop, inplace=True)
    
    print(f"✅ Dataset 'dataset_completo.csv' carregado. Total de {len(df_events)} eventos 'bstop_open'.")
    print("   Timestamp convertido para 'America/Sao_Paulo'.")
    print("\n📊 Preview dos dados:")
    print(df_events.head())
    
except FileNotFoundError:
    print("❌ Erro: Arquivo 'dataset_completo.csv' não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"❌ Ocorreu um erro ao carregar ou processar 'dataset_completo.csv': {e}")

### 3.2 Arquivos GTFS

In [ ]:
# Dicionário para armazenar os dataframes do GTFS
gtfs_data = {}
gtfs_files = [
    'sptrans/routes.txt', 
    'sptrans/trips.txt', 
    'sptrans/stops.txt', 
    'sptrans/stop_times.txt', 
    'sptrans/frequencies.txt'
]
gtfs_keys = [f.split('/')[-1].replace('.txt', '') for f in gtfs_files]

print("📥 Carregando arquivos GTFS...")

for file_path, key in zip(gtfs_files, gtfs_keys):
    try:
        gtfs_data[key] = pd.read_csv(
            file_path, 
            header=0, 
            engine='python',
            dtype=str
        )
        print(f"  ✅ Arquivo '{file_path}' carregado com sucesso.")
    
    except FileNotFoundError:
        print(f"  ❌ Erro: Arquivo '{file_path}' não encontrado.")
    except Exception as e:
        print(f"  ❌ Ocorreu um erro ao carregar '{file_path}': {e}")

# Variáveis individuais para facilitar o acesso
df_routes = gtfs_data.get('routes')
df_trips = gtfs_data.get('trips')
df_stops = gtfs_data.get('stops')
df_stop_times = gtfs_data.get('stop_times')
df_frequencies = gtfs_data.get('frequencies')

print(f"\n✅ {len(gtfs_data)} arquivos GTFS carregados.")

## 4. Limpeza e Extração de Coordenadas

### 4.1 Coordenadas do GTFS

In [ ]:
if df_stops is not None:
    try:
        # Converter colunas de lat/lon para numérico
        df_stops['stop_lat'] = pd.to_numeric(df_stops['stop_lat'])
        df_stops['stop_lon'] = pd.to_numeric(df_stops['stop_lon'])
        print("✅ Coordenadas de 'df_stops' convertidas para numérico.")
    except Exception as e:
        print(f"❌ Erro ao converter coordenadas de df_stops: {e}")
else:
    print("❌ ERRO: df_stops não está carregado.")

### 4.2 Extração de Coordenadas dos Eventos

In [ ]:
try:
    # Aplicar a função para extrair lat/lon
    df_events[['device_lat', 'device_lon']] = df_events['device_point'].apply(
        lambda x: pd.Series(extract_coords(x))
    )
    
    df_events[['stop_lat_event', 'stop_lon_event']] = df_events['ctm_stop_point'].apply(
        lambda x: pd.Series(extract_coords(x))
    )
    
    # Dropar colunas originais de POINT
    df_events.drop(columns=['device_point', 'ctm_stop_point'], inplace=True)
    print("✅ Coordenadas extraídas de 'df_events' (device e stop).")

except Exception as e:
    print(f"❌ Erro ao extrair coordenadas de df_events: {e}")

### 4.3 Cálculo de Distância Device-Stop

In [ ]:
def calculate_distance(row):
    """Calcula a distância geodésica entre device e stop"""
    if pd.isna(row['device_lat']) or pd.isna(row['stop_lat_event']):
        return np.nan
    
    device_coords = (row['device_lat'], row['device_lon'])
    stop_coords = (row['stop_lat_event'], row['stop_lon_event'])
    
    try:
        return geodesic(device_coords, stop_coords).meters
    except ValueError:
        return np.nan

print("\n🔄 Calculando 'dist_device_stop'...")
df_events['dist_device_stop'] = df_events.apply(calculate_distance, axis=1)
print("✅ Feature 'dist_device_stop' (em metros) criada.")
print("\n📊 Estatísticas de distância:")
print(df_events[['dist_device_stop']].describe())

### 4.4 Spatial Join - Encontrar gtfs_stop_id

In [ ]:
print("\n🔄 Iniciando 'spatial join' para encontrar o 'gtfs_stop_id' de cada evento...")

# 1. Preparar os dados de paradas do GTFS
df_stops_valid = df_stops.dropna(subset=['stop_lat', 'stop_lon']).copy()
df_stops_valid['stop_lat'] = df_stops_valid['stop_lat'].astype(float)
df_stops_valid['stop_lon'] = df_stops_valid['stop_lon'].astype(float)

stops_coords = df_stops_valid[['stop_lat', 'stop_lon']].values
print(f"  ✅ Criada árvore de referência com {len(stops_coords)} paradas do GTFS.")

# 2. Preparar os dados de eventos
df_events_valid = df_events.dropna(subset=['stop_lat_event', 'stop_lon_event']).copy()
events_coords = df_events_valid[['stop_lat_event', 'stop_lon_event']].values
print(f"  ✅ {len(events_coords)} eventos válidos para o matching.")

# 3. Construir a Árvore (cKDTree)
tree = cKDTree(stops_coords)

# 4. Consultar a Árvore
print("  🔄 Buscando paradas mais próximas (Nearest Neighbors)...")
distances, idxs = tree.query(events_coords, k=1)
print("  ✅ Busca concluída.")

# 5. Mapear os 'stop_id' de volta para os eventos
matched_stop_ids = df_stops_valid.iloc[idxs]['stop_id'].values
df_events_valid['gtfs_stop_id'] = matched_stop_ids

# 6. Juntar o 'gtfs_stop_id' de volta ao DataFrame original
df_events = df_events.merge(
    df_events_valid[['gtfs_stop_id']], 
    left_index=True, 
    right_index=True, 
    how='left'
)

print("\n✅ 'gtfs_stop_id' adicionado ao df_events.")
print(f"   Total de eventos com 'gtfs_stop_id' mapeado: {df_events['gtfs_stop_id'].notnull().sum()}")
print(f"   Total de eventos sem 'gtfs_stop_id': {df_events['gtfs_stop_id'].isnull().sum()}")

print("\n📊 Preview de df_events com 'gtfs_stop_id':")
print(df_events.head())

## 5. Criação da Variável Alvo (Target)

### 5.1 Proxy de Lotação

In [ ]:
print("\n🔄 Iniciando a criação da variável alvo (lotacao_proxy)...")

# Janela de agregação: 2 minutos
agg_window = '2T'

# Agrupar e contar usuários únicos
df_events_indexed = df_events.set_index('event_timestamp')

df_counts = df_events_indexed.groupby(
    ['gtfs_stop_id', pd.Grouper(freq=agg_window)]
)['user_pseudo_id'].nunique()

df_proxy = df_counts.to_frame(name='user_count_5min')
df_proxy.reset_index(inplace=True)

print(f"  ✅ Agregação de {len(df_proxy)} janelas de {agg_window} por parada concluída.")

# Definir thresholds e criar classes
bins = [0, 1, 2, np.inf]
labels = ['Baixa', 'Média', 'Alta']

df_proxy['lotacao_proxy'] = pd.cut(
    df_proxy['user_count_5min'], 
    bins=bins, 
    labels=labels, 
    right=True
)
print("  ✅ Classes 'Baixa', 'Média', 'Alta' definidas.")

# Juntar o proxy de volta ao DataFrame original
df_events['time_bin'] = df_events['event_timestamp'].dt.floor(agg_window)
df_proxy.rename(columns={'event_timestamp': 'time_bin'}, inplace=True)

df_final = df_events.merge(
    df_proxy,
    on=['time_bin', 'gtfs_stop_id'],
    how='left'
)

print("  ✅ Variável 'lotacao_proxy' juntada ao dataset principal.")

print("\n📊 Distribuição da Variável Alvo (lotacao_proxy):")
print(df_final['lotacao_proxy'].value_counts(normalize=True) * 100)

print(f"\n⚠️  Eventos sem proxy (nulos): {df_final['lotacao_proxy'].isnull().sum()}")

df_final.drop(columns=['time_bin', 'user_count_5min'], inplace=True, errors='ignore')

print("\n📊 Preview do df_final:")
print(df_final.head())

### 5.2 Conversão para Classificação Binária

In [ ]:
print("\n🔄 Adaptando para classificação binária (Detecção de Evento Raro)...")

# Mapear classes para binária
df_final['lotacao_proxy_binaria'] = df_final['lotacao_proxy'].map({
    'Baixa': 'Baixa',
    'Média': 'Nao_Baixa',
    'Alta': 'Nao_Baixa'
})

print("\n📊 Distribuição da Nova Variável Alvo (Binária):")
print(df_final['lotacao_proxy_binaria'].value_counts(normalize=True) * 100)

print("\n📊 Contagem Absoluta (Binária):")
print(df_final['lotacao_proxy_binaria'].value_counts())

# Remover a coluna proxy original de 3 classes
df_final.drop(columns=['lotacao_proxy'], inplace=True)

print("\n✅ Variável alvo re-processada para classificação binária.")

## 6. Engenharia de Features

### 6.1 Features Temporais

In [ ]:
print("\n🔄 Criando features temporais...")

# Garantir que o timestamp é o índice
if 'event_timestamp' in df_final.columns:
    df_final.set_index('event_timestamp', inplace=True)

# Features de Tempo
df_final['time_hour'] = df_final.index.hour
df_final['time_day_of_week'] = df_final.index.dayofweek  # Segunda=0, Domingo=6
df_final['time_day_of_month'] = df_final.index.day
df_final['time_month'] = df_final.index.month

# É Feriado?
df_final['is_holiday'] = pd.Series(df_final.index.date).apply(lambda x: x in br_holidays).values
df_final['is_holiday'] = df_final['is_holiday'].astype(int)

# É Fim de Semana?
df_final['is_weekend'] = (df_final['time_day_of_week'] >= 5).astype(int)  # 5=Sábado, 6=Domingo

# É Hora de Pico?
def is_peak(hour):
    if (hour >= 6 and hour < 9) or (hour >= 17 and hour < 19):
        return 1
    return 0

df_final['is_peak_hour'] = df_final['time_hour'].apply(is_peak)

# Resetar o índice
df_final.reset_index(inplace=True)

print("✅ Features temporais criadas:")
print("   - time_hour, time_day_of_week, time_day_of_month, time_month")
print("   - is_holiday, is_weekend, is_peak_hour")

print("\n📊 Preview:")
print(df_final[['event_timestamp', 'time_hour', 'time_day_of_week', 'is_holiday', 'is_peak_hour']].head())

### 6.2 Features de Serviço (Headway do GTFS)

In [ ]:
print("\n🔄 Criando features de headway (frequência de ônibus)...")

try:
    # 1. Limpar e Converter Frequencies
    df_freq = df_frequencies.copy()
    df_freq['headway_secs'] = pd.to_numeric(df_freq['headway_secs'])
    df_freq['start_hour'] = df_freq['start_time'].str.split(':').str[0].astype(int)
    
    # 2. Limpar Stop Times
    df_st = df_stop_times.copy()
    
    # 3. Juntar Frequências com Stop Times
    print("  🔄 Iniciando merge (stop_times + frequencies)...")
    df_freq_merged = df_st[['trip_id', 'stop_id']].merge(
        df_freq[['trip_id', 'start_hour', 'headway_secs']],
        on='trip_id',
        how='inner'
    )
    print("  ✅ Merge concluído.")
    
    df_freq_merged = df_freq_merged.drop_duplicates(subset=['stop_id', 'start_hour', 'trip_id'])
    
    # 4. Calcular Headway Médio por Parada/Hora
    print("  🔄 Calculando headway médio por parada/hora...")
    df_headway_avg = df_freq_merged.groupby(
        ['stop_id', 'start_hour']
    )['headway_secs'].mean().to_frame(name='headway_avg_stop_hour')
    
    df_headway_avg.reset_index(inplace=True)
    
    print(f"  ✅ Headway médio calculado para {len(df_headway_avg)} combinações.")

    # 5. Juntar ao df_final
    print("  🔄 Juntando feature 'headway' ao dataset final...")
    df_final = df_final.merge(
        df_headway_avg,
        left_on=['gtfs_stop_id', 'time_hour'],
        right_on=['stop_id', 'start_hour'],
        how='left'
    )
    
    df_final.drop(columns=['stop_id', 'start_hour'], inplace=True, errors='ignore')
    
    # Preencher nulos (paradas sem frequência) com valor alto (baixa frequência)
    headway_nulos = df_final['headway_avg_stop_hour'].isnull().sum()
    print(f"  ℹ️  {headway_nulos} eventos sem headway (preenchidos com 3600s).")
    df_final['headway_avg_stop_hour'].fillna(3600, inplace=True)
    
    print("  ✅ Feature 'headway_avg_stop_hour' criada.")

except Exception as e:
    print(f"  ⚠️  Erro ao processar headway: {e}")
    print("  ℹ️  Criando feature 'headway_avg_stop_hour' vazia (3600s).")
    df_final['headway_avg_stop_hour'] = 3600

### 6.3 Features Cíclicas (Temporal)

In [ ]:
print("\n🔄 Criando features cíclicas...")

# Features cíclicas para hora
df_final['hour_sin'] = np.sin(2 * np.pi * df_final['time_hour'] / 24)
df_final['hour_cos'] = np.cos(2 * np.pi * df_final['time_hour'] / 24)

# Features cíclicas para dia da semana
df_final['day_sin'] = np.sin(2 * np.pi * df_final['time_day_of_week'] / 7)
df_final['day_cos'] = np.cos(2 * np.pi * df_final['time_day_of_week'] / 7)

print("✅ Features cíclicas criadas:")
print("   - hour_sin, hour_cos")
print("   - day_sin, day_cos")

### 6.4 Features de Interação

In [ ]:
print("\n🔄 Criando features de interação...")

# Interações entre features
df_final['headway_x_hour'] = df_final['headway_avg_stop_hour'] * df_final['time_hour']
df_final['headway_x_weekend'] = df_final['headway_avg_stop_hour'] * df_final['is_weekend']
df_final['dist_x_peak'] = df_final['dist_device_stop'] * df_final['is_peak_hour']
df_final['dist_x_weekend'] = df_final['dist_device_stop'] * df_final['is_weekend']

print("✅ Features de interação criadas:")
print("   - headway_x_hour, headway_x_weekend")
print("   - dist_x_peak, dist_x_weekend")

### 6.5 Features de Agregação por Parada

In [ ]:
print("\n🔄 Criando features de agregação por parada...")

# 1. Taxa de eventos por parada
stop_event_rate = df_final.groupby('gtfs_stop_id')['lotacao_proxy_binaria'].agg([
    ('stop_event_rate', lambda x: (x == 'Nao_Baixa').mean()),
    ('stop_event_count', lambda x: (x == 'Nao_Baixa').sum()),
    ('stop_total_samples', 'count')
]).reset_index()

df_final = df_final.merge(stop_event_rate, on='gtfs_stop_id', how='left')

# 2. Estatísticas de distância por parada
stop_dist_stats = df_final.groupby('gtfs_stop_id')['dist_device_stop'].agg([
    ('stop_dist_mean', 'mean'),
    ('stop_dist_std', 'std')
]).reset_index()

df_final = df_final.merge(stop_dist_stats, on='gtfs_stop_id', how='left')

# 3. Estatísticas de headway por parada
stop_headway_stats = df_final.groupby('gtfs_stop_id')['headway_avg_stop_hour'].agg([
    ('stop_headway_mean', 'mean'),
    ('stop_headway_std', 'std')
]).reset_index()

df_final = df_final.merge(stop_headway_stats, on='gtfs_stop_id', how='left')

# Preencher NaN em std
df_final['stop_dist_std'].fillna(0, inplace=True)
df_final['stop_headway_std'].fillna(0, inplace=True)

print("✅ Features de agregação criadas:")
print("   - stop_event_rate, stop_event_count, stop_total_samples")
print("   - stop_dist_mean, stop_dist_std")
print("   - stop_headway_mean, stop_headway_std")

print("\n📊 Preview:")
print(df_final[['gtfs_stop_id', 'stop_event_rate', 'stop_dist_mean', 'stop_headway_mean']].head())

## 7. Preparação Final e Encoding

In [ ]:
print("\n🔄 Preparando dataset final...")

# 1. Remover linhas sem rótulo
nulos_antes = df_final['lotacao_proxy_binaria'].isnull().sum()
if nulos_antes > 0:
    df_final.dropna(subset=['lotacao_proxy_binaria'], inplace=True)
    print(f"  ℹ️  {nulos_antes} linhas sem rótulo removidas.")

# 2. Encoding da Variável Alvo
target_map = {'Baixa': 0, 'Nao_Baixa': 1}
df_final['target'] = df_final['lotacao_proxy_binaria'].map(target_map)
print("  ✅ Variável alvo mapeada: 'Baixa'=0, 'Nao_Baixa'=1")

# 3. Encoding de Variáveis Categóricas
categorical_features = ['gtfs_stop_id', 'time_day_of_week', 'time_hour']
for col in categorical_features:
    le = LabelEncoder()
    df_final[col] = le.fit_transform(df_final[col].astype(str))
    print(f"  ✅ Feature '{col}' codificada como categórica.")

# 4. Ordenar por tempo (CRUCIAL para validação temporal)
df_final.sort_values('event_timestamp', inplace=True)

print(f"\n✅ Dataset final preparado!")
print(f"   Total de amostras: {len(df_final)}")
print(f"   Eventos positivos (1): {df_final['target'].sum()} ({df_final['target'].mean()*100:.2f}%)")
print(f"   Total de features: {len(df_final.columns)}")

## 8. Exportação da Tabela Final

In [ ]:
print("\n💾 Exportando dataset processado...")

# Salvar em CSV
output_file = 'dataset_cittamobi_final.csv'
df_final.to_csv(output_file, index=False)

print(f"✅ Dataset salvo em: {output_file}")
print(f"   Tamanho: {len(df_final)} linhas x {len(df_final.columns)} colunas")

# Resumo das features criadas
print("\n" + "="*70)
print("📋 RESUMO DAS FEATURES CRIADAS")
print("="*70)

print("\n1️⃣  Features Temporais (11):")
print("   - time_hour, time_day_of_week, time_day_of_month, time_month")
print("   - is_holiday, is_weekend, is_peak_hour")
print("   - hour_sin, hour_cos, day_sin, day_cos (cíclicas)")

print("\n2️⃣  Features Geoespaciais (5):")
print("   - dist_device_stop (distância em metros)")
print("   - device_lat, device_lon, stop_lat_event, stop_lon_event")

print("\n3️⃣  Features de Serviço (GTFS) (2):")
print("   - gtfs_stop_id (ID da parada mais próxima)")
print("   - headway_avg_stop_hour (frequência média de ônibus)")

print("\n4️⃣  Features de Interação (4):")
print("   - headway_x_hour, headway_x_weekend")
print("   - dist_x_peak, dist_x_weekend")

print("\n5️⃣  Features de Agregação por Parada (7):")
print("   - stop_event_rate, stop_event_count, stop_total_samples")
print("   - stop_dist_mean, stop_dist_std")
print("   - stop_headway_mean, stop_headway_std")

print("\n6️⃣  Variável Alvo (2):")
print("   - target (0=Baixa, 1=Nao_Baixa)")
print("   - lotacao_proxy_binaria (categoria original)")

print("\n" + "="*70)
print("🎉 ETL PIPELINE CONCLUÍDO COM SUCESSO!")
print("="*70)
print("O dataset está pronto para treinamento de modelos de ML.")

## 9. Verificação Final

In [ ]:
print("\n" + "="*70)
print("🔍 VERIFICAÇÃO FINAL DO DATASET")
print("="*70)

print("\n📊 Informações Gerais:")
print(df_final.info())

print("\n📈 Estatísticas Descritivas:")
print(df_final.describe())

print("\n🎯 Distribuição do Target:")
print(df_final['target'].value_counts())
print(f"\nBalanceamento:")
print(df_final['target'].value_counts(normalize=True) * 100)

print("\n⚠️  Valores Nulos por Coluna:")
nulos = df_final.isnull().sum()
if nulos.sum() > 0:
    print(nulos[nulos > 0])
else:
    print("   Nenhum valor nulo encontrado! ✅")

print("\n📋 Preview do Dataset Final:")
print(df_final.head(10))

print("\n✅ Verificação concluída!")